# Generating OncoPrint Data Files

The script will process all variant files and output files in an ingestible format for the R OncoPrint function.

It will output oncoprint data for both replicate files and the merged variant callsets.

In [1]:
import os
import pandas as pd

In [2]:
# Load all cosmic variants called in this dataset
# This file was generated in filter_variants.ipynb
cosmic_all_file = os.path.join('results', 'all_cosmic_variants.tsv')
cosmic_all_df = pd.read_table(cosmic_all_file)
cosmic_all_df.head(3)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,phyloP20way_mammalian,phastCons7way_vertebrate,phastCons20way_mammalian,SiPhy_29way_logOdds,Otherinfo,final_id,sample_name,het,quality,depth
0,1,874779,874826,CCTCCCCAGCCACGGTGAGGACCCACCCTGGCATGATCCCCCTCATCA,-,exonic,SAMD11,.,nonframeshift deletion,SAMD11:NM_152486:exon7:c.645_692del:p.G220Dfs*447,...,.,.,.,.,het\t.\t20,008-M2-F0,019-F0,het,.,20
1,1,26510311,26510311,C,-,exonic,CNKSR1,.,frameshift deletion,CNKSR1:NM_001297647:exon9:c.866delC:p.P291Hfs*...,...,.,.,.,.,het\t.\t31,008-M2-F0,019-F0,het,.,31
2,1,158576883,158576883,G,A,exonic,OR10Z1,.,nonsynonymous SNV,OR10Z1:NM_001004478:exon1:c.G655A:p.A219T,...,0.949,0.822,0.761,4.671,het\t.\t127,008-M2-F0,019-F0,het,.,127


In [3]:
# What are the 50 most commonly altered COSMIC genes?
top_n = 50
paad_genes = cosmic_all_df['Gene.refGene'].value_counts().head(top_n).index.tolist()
cosmic_all_df['Gene.refGene'].value_counts().head(20)

NCOR2                39
OR11G2               29
KRAS                 29
MUC4;MUC4            28
DNHD1                27
TSPYL1               26
CBWD1                25
GOLGA8M              24
MARF1                24
ARHGEF11;ETV3L       24
WDR66                24
PHF2                 24
LCE4A                23
GOLGA6D              22
KRTAP5-11;FAM86C1    21
CGB8                 21
ACIN1                21
POTEI                21
MROH8;MROH8          20
BRCA1                20
Name: Gene.refGene, dtype: int64

## Generate OncoPrint Data

### For All Replicates

In [4]:
%%time

# Process each replicate by observed COSMIC mutation# Proce 
variant_file_path = os.path.join('results', 'processed_vcfs')
variant_assign = []
case_id = []
for variant_file in os.listdir(variant_file_path):
    # Load and subset file to only variants in the COSMIC db
    variant_df = pd.read_table(os.path.join(variant_file_path, variant_file), index_col=0)
    variant_sub_df = variant_df[variant_df['cosmic70'] != '.']
    
    # Define mutated genes if they exist for the given variant
    variant_class = ['MUT;' if x in variant_sub_df['Gene.refGene'].tolist() else ''
                     for x in paad_genes]
    
    # Store results
    variant_assign.append(variant_class)
    case_id.append(variant_file.replace('_001_processed_variants.tsv.bz2', ''))

CPU times: user 15.7 s, sys: 472 ms, total: 16.2 s
Wall time: 16.2 s


In [5]:
# Generate and save oncoprint data for all replicates
oncoprint_file = os.path.join('results', 'oncoprint_replicates.tsv')
oncoprint_df = pd.DataFrame(variant_assign, index=case_id, columns=paad_genes).sort_index()
oncoprint_df.index.name = 'Case.ID'
oncoprint_df.to_csv(oncoprint_file, sep='\t')

In [6]:
oncoprint_df.head(3)

,NCOR2,OR11G2,KRAS,MUC4;MUC4,DNHD1,TSPYL1,CBWD1,GOLGA8M,MARF1,ARHGEF11;ETV3L,...,ZNF676,DPCR1,TP53,FAM161B,GNAS,SMAD4,AGFG1,PIK3CA,PRELID3B,ADRA2B;ADRA2B
Case.ID,,,,,,,,,,,,,,,,,,,,,
001-F0_S3_L001,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,,MUT;,MUT;,...,,,,MUT;,,,,,MUT;,
001-F0_S3_L002,,MUT;,MUT;,,MUT;,MUT;,MUT;,,MUT;,MUT;,...,,MUT;,,MUT;,,MUT;,,,,
001-F0_S3_L003,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,,MUT;,MUT;,...,,MUT;,,MUT;,,MUT;,,,MUT;,


### Merged Samples

In [7]:
variant_assign_consensus = []
case_id_consensus = []
for sample_id in set(cosmic_all_df['final_id']):
    # Subset file to given sample ID
    variant_sub_df = cosmic_all_df.query('final_id == @sample_id')
    
    # Define mutated genes if they exist for the given variant
    variant_class = ['MUT;' if x in variant_sub_df['Gene.refGene'].tolist() else ''
                     for x in paad_genes]
    
    # Store results
    variant_assign_consensus.append(variant_class)
    case_id_consensus.append(sample_id)

In [8]:
# Generate and save oncoprint data for consensus samples
oncoprint_consensus_file = os.path.join('results', 'oncoprint_merged.tsv')

oncoprint_consensus_df = (
    pd.DataFrame(variant_assign_consensus,
                 index=case_id_consensus,
                 columns=paad_genes)
    )
oncoprint_consensus_df.index.name = 'Case.ID'
oncoprint_consensus_df.to_csv(oncoprint_consensus_file, sep='\t')

In [9]:
oncoprint_consensus_df.head(3)

,NCOR2,OR11G2,KRAS,MUC4;MUC4,DNHD1,TSPYL1,CBWD1,GOLGA8M,MARF1,ARHGEF11;ETV3L,...,ZNF676,DPCR1,TP53,FAM161B,GNAS,SMAD4,AGFG1,PIK3CA,PRELID3B,ADRA2B;ADRA2B
Case.ID,,,,,,,,,,,,,,,,,,,,,
006-F5,MUT;,MUT;,MUT;,,MUT;,,MUT;,MUT;,MUT;,MUT;,...,,,,,,MUT;,,MUT;,MUT;,MUT;
048-M1-F5,MUT;,MUT;,MUT;,MUT;,,MUT;,,,MUT;,MUT;,...,,,MUT;,,MUT;,MUT;,,,,
008-F0,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,MUT;,...,,MUT;,,MUT;,MUT;,,MUT;,,,


## COSMIC Mutational Similarity

Output mutational similarity data for all replicates and consensus samples. The COSMIC mutational similarity is built from a (0,1) sample by COSMIC mutation matrix.

### For All Replicates

In [10]:
# How many COSMIC mutation IDs are in the entire set and how many are unique?
print('All COSMIC mutations: {}'.format(cosmic_all_df.shape[0]))
unique_cosmic_ids = set(cosmic_all_df['cosmic70'])
print('Unique COSMIC mutations: {}'.format(len(unique_cosmic_ids)))

All COSMIC mutations: 3259
Unique COSMIC mutations: 1141


In [11]:
case_id = []
cosmic_similarity_list = []
for variant_file in os.listdir(variant_file_path):
    # Load and subset file to only variants in the COSMIC db
    variant_df = pd.read_table(os.path.join(variant_file_path, variant_file), index_col=0)
    variant_sub_df = variant_df[variant_df['cosmic70'] != '.']
    
    # Define membership in COSMIC IDs
    cosmic_class = [1 if x in variant_sub_df['cosmic70'].tolist() else 0
                    for x in unique_cosmic_ids]
    
    # Store results
    cosmic_similarity_list.append(cosmic_class)
    case_id.append(variant_file.replace('_001_processed_variants.tsv.bz2', ''))

In [12]:
# Generate COSMIC id membership data (for downstream similarity matrix)
cosmic_common_file = os.path.join('results', 'cosmic_similarity_replicates.tsv')
cosmic_common_df = (
    pd.DataFrame(cosmic_similarity_list,
                 index=case_id,
                 columns=unique_cosmic_ids)
    )
cosmic_common_df.index.name = 'Case.ID'
cosmic_common_df.to_csv(cosmic_common_file, sep='\t')

### Consensus samples

In [13]:
case_id_consensus = []
cosmic_similarity_consensus_list = []
for sample_id in set(cosmic_all_df['final_id']):
    # Subset file to given sample ID
    variant_sub_df = cosmic_all_df.query('final_id == @sample_id')
    
    # Define membership in COSMIC IDs
    cosmic_class = [1 if x in variant_sub_df['cosmic70'].tolist() else 0
                    for x in unique_cosmic_ids]
    
    # Store results
    cosmic_similarity_consensus_list.append(cosmic_class)
    case_id_consensus.append(sample_id)

In [14]:
common_cosmic_consensus_file = os.path.join('results', 'cosmic_similarity_merged.tsv')
cosmic_common_consensus_df = pd.DataFrame(cosmic_similarity_consensus_list,
                                          index=case_id_consensus,
                                          columns=unique_cosmic_ids)
cosmic_common_consensus_df.index.name = 'Case.ID'
cosmic_common_consensus_df.to_csv(common_cosmic_consensus_file, sep='\t')

## What about prefiltered variants (i.e. before COSMIC filtering)

Observed merged samples with cosmic similarity only

In [15]:
# Load all prefiltered cosmic variants called in this dataset
# This file was generated in filter_variants.ipynb
file = os.path.join('results', 'all_cosmic_prefiltered_variants.tsv')
cosmic_prefiltered_df = pd.read_table(file)
cosmic_prefiltered_df.head(3)

/home/gway/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,phyloP20way_mammalian,phastCons7way_vertebrate,phastCons20way_mammalian,SiPhy_29way_logOdds,Otherinfo,final_id,sample_name,het,quality,depth
0,1,17379,17379,G,A,ncRNA_exonic,MIR6859-1;MIR6859-2;MIR6859-3;MIR6859-4,.,.,.,...,.,.,.,.,het\t.\t34,008-M2-F0,019-F0,het,.,34
1,1,139058,139058,C,G,ncRNA_exonic,LOC729737,.,.,.,...,.,.,.,.,het\t.\t125,008-M2-F0,019-F0,het,.,125
2,1,664844,664844,T,C,ncRNA_exonic,LOC100133331,.,.,.,...,.,.,.,.,het\t.\t14,008-M2-F0,019-F0,het,.,14


In [16]:
paad_prefiltered_genes = cosmic_prefiltered_df['Gene.refGene'].value_counts().head(top_n).index.tolist()
cosmic_prefiltered_df['Gene.refGene'].value_counts().head(20)

NONE;NONE                5484
NBPF25P                   963
OR4E1;LINC02332           938
DNAH5                     858
RYR2                      732
SYNE2                     719
MIR1268A                  696
PIKFYVE                   685
TG                        651
CUBN                      621
LRP1B                     618
CNTNAP3P2;XLOC_007697     614
LOC100996724              600
OR4A47;TRIM49B            586
CSMD3                     584
NONE;LOC654342            581
RYR3                      577
SYNE1                     571
LOC440040;OR4C13          563
NONE;CHEK2P2              560
Name: Gene.refGene, dtype: int64

In [28]:
# Only consider mutations that change amino acid sequence
cosmic_prefiltered_df = cosmic_prefiltered_df[cosmic_prefiltered_df['AAChange.refGene'] != '.']

In [29]:
# How many prefiltered COSMIC mutation IDs are in the entire set and how many are unique?
print('All prefiltered COSMIC mutations: {}'.format(cosmic_prefiltered_df.shape[0]))
unique_mutations = set(cosmic_prefiltered_df['AAChange.refGene'])
print('Unique nonsilent mutations: {}'.format(len(unique_mutations)))

All prefiltered COSMIC mutations: 56516
Unique nonsilent mutations: 29191


In [33]:
case_id_consensus = []
cosmic_similarity_consensus_list = []
for sample_id in set(cosmic_prefiltered_df['final_id']):
    # Subset file to given sample ID
    variant_sub_df = cosmic_prefiltered_df.query('final_id == @sample_id')
    
    # Define membership in COSMIC IDs
    cosmic_class = [1 if x in variant_sub_df['AAChange.refGene'].tolist() else 0
                    for x in unique_mutations]
    
    # Store results
    cosmic_similarity_consensus_list.append(cosmic_class)
    case_id_consensus.append(sample_id)

In [34]:
common_cosmic_consensus_file = os.path.join('results', 'cosmic_prefiltered_similarity_merged.tsv')
cosmic_common_consensus_df = pd.DataFrame(cosmic_similarity_consensus_list,
                                          index=case_id_consensus,
                                          columns=unique_mutations)
cosmic_common_consensus_df.index.name = 'Case.ID'
cosmic_common_consensus_df.to_csv(common_cosmic_consensus_file, sep='\t')